### All neccessary imports

In [3]:
!pip install praw

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.3/189.3 kB 8.6 MB/s eta 0:00:00


In [4]:
import numpy as np
import pandas as pd
import re
import nltk
import praw
import tensorflow as tf

### Basic Labelling

In [5]:
# TextBlob Sentiment Labeling Function
def textblob_labeler(text):
    analysis = TextBlob(text)
    if analysis.sentiment.polarity > 0.1:
        return 'positive'
    elif analysis.sentiment.polarity < -0.1:
        return 'negative'
    else:
        return 'neutral'

In [8]:
# Data Collection
import praw

# Replace 'YOUR_USER_AGENT' with a descriptive string
user_agent = 'script:my_reddit_scraper:v1.0 (by )'

# Data Collection
def fetch_reddit_data(subreddit, count=100):
    reddit = praw.Reddit(client_id='', client_secret='', user_agent=user_agent)
    posts = [post.title for post in reddit.subreddit(subreddit).hot(limit=count)]
    return posts

def fetch_reddit_post_comments(post_id, count=100):
    reddit = praw.Reddit(client_id='', client_secret='', user_agent=user_agent)
    submission = reddit.submission(id=post_id)
    comments = [comment.body for comment in submission.comments if hasattr(comment, "body")][:count]
    return comments


In [9]:
# User Input
subreddit_name = input("Enter the subreddit name or leave blank for specific post: ")
reddit_post_id = input("Enter the Reddit post ID (if fetching comments from a post): ")

Enter the subreddit name or leave blank for specific post: r/TeenIndia
Enter the Reddit post ID (if fetching comments from a post): 


In [10]:
# Data Collection
data = {"text": []}

if subreddit_name:
    data["text"] += fetch_reddit_data(subreddit_name, 50)
if reddit_post_id:
    data["text"] += fetch_reddit_post_comments(reddit_post_id, 50)


It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



BadRequest: received 400 HTTP response

In [ ]:
# Create DataFrame and Generate Labels
df = pd.DataFrame(data)
df['sentiment'] = df['text'].apply(textblob_labeler)  # Apply TextBlob labeling

In [ ]:
# Data Cleaning
def clean_text(text):
    text = text.lower()
    text = re.sub(r'http\S+', '', text)  # Remove URLs
    text = re.sub(r'@\w+', '', text)  # Remove mentions
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # Remove special characters
    return text

df['cleaned_text'] = df['text'].apply(clean_text)


In [ ]:
# Tokenization and Padding
tokenizer = Tokenizer(num_words=5000, oov_token="<OOV>")
tokenizer.fit_on_texts(df['cleaned_text'])
sequences = tokenizer.texts_to_sequences(df['cleaned_text'])
padded_sequences = pad_sequences(sequences, maxlen=50, padding='post', truncating='post')

In [ ]:
# Label Encoding
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(df['sentiment'])

In [ ]:
# Splitting Data
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, labels, test_size=0.2, random_state=42)

In [ ]:
# Model Building
model = Sequential([
    Embedding(input_dim=5000, output_dim=64, input_length=50),
    LSTM(64, return_sequences=True),
    Dropout(0.2),
    LSTM(32),
    Dense(32, activation='relu'),
    Dropout(0.2),
    Dense(3, activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# Model Training
model.fit(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

# Evaluation
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {accuracy * 100:.2f}%')